In [ ]:
#pip install scikit-learn
#pip install catboost
#pip install OS
#pip install numpy
#pip install pandas
#pip install openpyxl

  Using cached scikit_learn-1.6.1-cp310-cp310-win_amd64.whl (11.1 MB)
  Using cached joblib-1.5.1-py3-none-any.whl (307 kB)
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\tkfkg\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [4]:
# 📌 1. 기본 라이브러리 불러오기
import pandas as pd
import numpy as np
import os
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder

In [ ]:
# 📌 2. 연도별 데이터 로드
data_dir = '../../Data'  # 경로는 프로젝트 구조에 따라 맞게 설정
years = list(range(2019, 2024))

dfs = []
for year in years:
    file = os.path.join(data_dir, 'Trading_Area', f'Trading_Area_{year}.csv')
    if os.path.exists(file):
        try:
            df = pd.read_csv(file, encoding='utf-8')
        except UnicodeDecodeError:
            df = pd.read_csv(file, encoding='cp949')
        df['연도'] = year
        if '행정동코드' not in df.columns:
            if '행정동_코드' in df.columns:
                df['행정동_코드'] = df['행정동_코드'].astype(str).str.zfill(8)
            else:
                raise KeyError(f'파일 {file}에서 행정동코드 관련 컬럼을 찾을 수 없습니다.')
        dfs.append(df)

people_df = pd.read_csv(os.path.join(data_dir, 'CompanyPeople.csv'), encoding='cp949')
people_df['행정동_코드'] = people_df['행정동_코드'].astype(str).str.zfill(8)

dong_map = pd.read_excel(os.path.join(data_dir, '행정동코드_매핑정보.xlsx'), header=1)
dong_map = dong_map[['H_DNG_CD', 'H_DNG_NM']].rename(columns={'H_DNG_CD':'행정동코드', 'H_DNG_NM':'행정동명'})
dong_map['행정동코드'] = dong_map['행정동코드'].astype(str).str.zfill(8)

# 📌 3. 모든 연도별 데이터 하나로 합치기
data = pd.concat(dfs, ignore_index=True)

# 📌 4. 직장인구 데이터 병합
data = data.merge(people_df, on=['기준_년분기_코드', '행정동_코드'], how='left')

# 📌 5. 결측치 처리 (평균 대체)
imputer = SimpleImputer(strategy='mean')
data_num = data.select_dtypes(include=[np.number])
data[data_num.columns] = imputer.fit_transform(data_num)

# 📌 6. 범주형 변수 처리
le_dong = LabelEncoder()
le_biz = LabelEncoder()
data['행정동코드_le'] = le_dong.fit_transform(data['행정동_코드'])
data['업종_le'] = le_biz.fit_transform(data['서비스_업종_코드_명'])

# 📌 7. 학습 데이터 준비
X = data[['연도', '행정동코드_le', '업종_le']]
y = data['당월_매출_금액']

# 📌 8. RandomForestRegressor 학습
model = RandomForestRegressor(
    n_estimators=300,  # 트리 개수
    max_depth=15,
    random_state=42,
    n_jobs=-1
)
model.fit(X, y)

# 📌 9. 2025년 예측 데이터 준비 (모든 (행정동코드, 업종) 조합)
dong_list = data['행정동_코드'].unique()
biz_list = data['서비스_업종_코드_명'].unique()

pred_df = pd.DataFrame([
    {'연도': 2025, '행정동코드': d, '업종': b}
    for d in dong_list for b in biz_list
])

# 📌 10. 범주형 변수 인코딩
pred_df['행정동코드_le'] = le_dong.transform(pred_df['행정동코드'])
pred_df['업종_le'] = le_biz.transform(pred_df['업종'])

# 📌 11. 직장인구 데이터 병합 (2024년 데이터 사용 가정)
people_2024 = people_df[people_df['기준_년분기_코드'] == 2024].drop(columns=['기준_년분기_코드'])

# people_df 컬럼명 통일
if '행정동_코드' in people_df.columns:
    people_df.rename(columns={'행정동_코드': '행정동코드'}, inplace=True)

# 2024년 데이터 준비
people_2024 = people_df[people_df['기준_년분기_코드'] == 2024].drop(columns=['기준_년분기_코드'])

# ✅ 병합: 행정동코드 기준으로 정확히!
pred_df = pred_df.merge(people_2024, on='행정동코드', how='left')
pred_df.fillna(people_2024.mean(numeric_only=True), inplace=True)

# 📌 12. 예측
X_pred = pred_df[['연도', '행정동코드_le', '업종_le']]
pred_df['예측_총매출'] = model.predict(X_pred)

# 📌 13. 행정동명 붙이기
pred_df = pred_df.merge(dong_map, on='행정동코드', how='left')

# 📌 14. 순위 계산
pred_df['순위'] = pred_df.groupby('행정동코드')['예측_총매출'].rank(ascending=False, method='min')
pred_df['연도'] = 2025

# 📌 15. 최종 정리 및 저장
final = pred_df[['연도', '행정동코드', '행정동명', '업종', '예측_총매출', '순위']]
final = final.sort_values(by=['행정동코드', '순위']).reset_index(drop=True)

final.to_csv('./Predicted_2025_Top_Business_Company.csv', index=False, encoding='utf-8-sig')

# 최종 결과 미리보기
final.head(10)


,연도,행정동코드,행정동명,업종,예측_총매출,순위
0,2025,11110515,청운효자동,조명용품,5.775176e+09,1.0
1,2025,11110515,청운효자동,커피-음료,4.306592e+09,2.0
2,2025,11110515,청운효자동,한식음식점,3.738617e+09,3.0
3,2025,11110515,청운효자동,서적,3.597058e+09,4.0
4,2025,11110515,청운효자동,양식음식점,2.744975e+09,5.0
5,2025,11110515,청운효자동,슈퍼마켓,2.040304e+09,6.0
6,2025,11110515,청운효자동,편의점,1.719586e+09,7.0
7,2025,11110515,청운효자동,의료기기,1.307927e+09,8.0
8,2025,11110515,청운효자동,컴퓨터및주변장치판매,1.212852e+09,9.0
9,2025,11110515,청운효자동,스포츠클럽,1.031641e+09,10.0


In [ ]:
import pandas as pd
from catboost import CatBoostRegressor, Pool

# 1. 기존 분석 결과 파일 합치기
years = list(range(2019, 2024))
df_list = []
for year in years:
    file = os.path.join(data_dir, f'{year}_sales.csv')
    if os.path.exists(file):
        try:
            df = pd.read_csv(file, encoding='utf-8')
        except UnicodeDecodeError:
            df = pd.read_csv(file, encoding='cp949')
        df['연도'] = year
        if '행정동코드' not in df.columns:
            if '행정동_코드' in df.columns:
                df['행정동_코드'] = df['행정동_코드'].astype(str).str.zfill(8)
            else:
                raise KeyError(f'파일 {file}에서 행정동코드 관련 컬럼을 찾을 수 없습니다.')
        dfs.append(df)


# 2. 행정동코드 매핑정보 로드
dong_map = pd.read_excel('../../Data/행정동코드_매핑정보.xlsx', header=1, engine='openpyxl')
dong_map = dong_map[['H_DNG_CD', 'H_DNG_NM']].rename(columns={'H_DNG_CD': '행정동코드', 'H_DNG_NM': '행정동명'})
dong_map['행정동코드'] = dong_map['행정동코드'].astype(str).str.zfill(8)

# 1️⃣ 컬럼명 통일
data = data.rename(columns={
    '당월_매출_금액': '총매출'  # 이 부분이 중요!
})

# 4️⃣ 전년도 매출 데이터와 merge해서 증감률 계산
data_prev = data.copy()
data_prev['연도'] += 1
data_prev = data_prev.rename(columns={'총매출': '총매출_prev'})

data_merged = data.merge(data_prev[['기준_년분기_코드', '행정동_코드', '서비스_업종_코드_명', '총매출_prev']],
                          on=['기준_년분기_코드', '행정동_코드', '서비스_업종_코드_명'], how='left')
data_merged['매출증감률'] = (data_merged['총매출'] - data_merged['총매출_prev']) / data_merged['총매출_prev']
data_merged['매출증감률'] = data_merged['매출증감률'].fillna(0)

# 3. 범주형 변수 처리
X = data_merged[['연도', '행정동_코드', '서비스_업종_코드_명']]
y = data_merged['총매출']
cat_features = ['행정동_코드', '서비스_업종_코드_명']

train_pool = Pool(X, y, cat_features=cat_features)

# 4. CatBoostRegressor 모델 학습
model = CatBoostRegressor(
    iterations=3000,
    learning_rate=0.01,
    depth=8,
    l2_leaf_reg=3,
    loss_function='RMSE',
    random_seed=42,
    cat_features=cat_features,
    task_type='GPU',  # GPU 사용 (없으면 'CPU'로 바꿔주세요)
    verbose=100
)
model.fit(train_pool)




0:	learn: 9993481164.7541695	total: 221ms	remaining: 11m 1s
100:	learn: 9608486570.2059059	total: 6.11s	remaining: 2m 55s
200:	learn: 9396529486.1952553	total: 12s	remaining: 2m 47s
300:	learn: 9225420797.3857040	total: 18s	remaining: 2m 41s
400:	learn: 9011574334.0982780	total: 23.9s	remaining: 2m 34s
500:	learn: 8898343370.5100136	total: 29.7s	remaining: 2m 28s
600:	learn: 8789411231.4412670	total: 35.3s	remaining: 2m 21s
700:	learn: 8705097999.5050926	total: 41s	remaining: 2m 14s
800:	learn: 8630754892.9278831	total: 46.6s	remaining: 2m 8s
900:	learn: 8575898801.5083399	total: 52.2s	remaining: 2m 1s
1000:	learn: 8513196902.6411648	total: 57.8s	remaining: 1m 55s
1100:	learn: 8481382743.9434233	total: 1m 3s	remaining: 1m 49s
1200:	learn: 8446234049.9579210	total: 1m 9s	remaining: 1m 43s
1300:	learn: 8390761224.3383198	total: 1m 14s	remaining: 1m 37s
1400:	learn: 8353813193.1701746	total: 1m 20s	remaining: 1m 31s
1500:	learn: 8319670708.8945866	total: 1m 26s	remaining: 1m 25s
1600:	lea

In [11]:
# 5. 2025년 예측 데이터 준비 (모든 (행정동코드, 업종) 조합)
dong_list = sorted(X['행정동_코드'].unique())
biz_list = sorted(X['서비스_업종_코드_명'].unique())
pred_df = pd.DataFrame([
    {"연도": 2025, "행정동_코드": d, "서비스_업종_코드_명": b}
    for d in dong_list for b in biz_list
])
pred_pool = Pool(pred_df, cat_features=cat_features)

# 6. 매출 예측
pred_df['예측_총매출'] = model.predict(pred_pool)


# 컬럼명 정리
pred_df = pred_df.rename(columns={'행정동_코드': '행정동코드', '서비스_업종_코드_명': '업종'})

# 7. 행정동명 병합
pred_df = pred_df.merge(dong_map, on='행정동코드', how='left')

# 7-1. 과거 평균 매출증감률 계산
avg_rate = data.groupby(['행정동_코드', '서비스_업종_코드_명'])['매출증감률'].mean().reset_index()
avg_rate.columns = ['행정동코드', '업종', '과거_평균_증감률']
pred_df = pred_df.merge(avg_rate, on=['행정동코드', '업종'], how='left')
pred_df['과거_평균_증감률'] = pred_df['과거_평균_증감률'].fillna(0)

# 7-2. 점수 계산 및 순위
pred_df['예측_점수'] = pred_df['예측_총매출'] * (1 + pred_df['과거_평균_증감률'])
pred_df['순위'] = pred_df.groupby('행정동코드')['예측_점수'].rank(ascending=False, method='min')
pred_df['연도'] = 2025

# 8. 최종 정리 및 CSV 저장
final = pred_df[['연도', '행정동코드', '행정동명', '업종', '예측_총매출', '순위']]
final = final.sort_values(by=['행정동코드', '순위']).reset_index(drop=True)
final.to_csv('./Predicted_2025_Top_Business.csv', index=False, encoding='utf-8-sig')

print("✅ 2025년 업종별 예측 결과 저장 완료! ./Predicted_2025_Top_Business.csv")

KeyError: 'Column not found: 매출증감률'